In [3]:
from IPython.display import Image, display
import PIL.Image
import io

import torch
import numpy as np
from processing_image import Preprocess
from modeling_frcnn import GeneralizedRCNN
from utils import Config
import utils


In [4]:
from datasets import load_dataset, Image
import pickle
import os
import cv2
import torch
import numpy as np
from torchvision import transforms

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
# load models and model components
frcnn_cfg = Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")
frcnn_cfg.model.DEVICE = device

frcnn = GeneralizedRCNN.from_pretrained("unc-nlp/frcnn-vg-finetuned", config=frcnn_cfg).to(device)

image_preprocess = Preprocess(frcnn_cfg)

# bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
# visualbert_vqa = VisualBertForQuestionAnswering.from_pretrained("uclanlp/visualbert-vqa")

loading configuration file cache
loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at /nas/home/bchidamb/.cache/torch/transformers/57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


In [7]:
cifar10_data = load_dataset("cifar10")

Found cached dataset cifar10 (/nas/home/bchidamb/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
train_data = cifar10_data["train"]
test_data = cifar10_data["test"]
to_tensor = transforms.ToTensor()

In [18]:
for idx, img_data in enumerate(train_data):
    print(img_data)
    if idx % 1000 == 0:
        print("On idx " + str(idx))
    image = np.array(img_data["img"])
    # run frcnn
    images, sizes, scales_yx = image_preprocess(image)
    output_dict = frcnn(
        images,
        sizes,
        scales_yx=scales_yx,
        padding="max_detections",
        max_detections=frcnn_cfg.max_detections,
        return_tensors="pt",
    )
    features = output_dict.get("roi_features")
    
#     visual_embeddings.append(features)
#     labels.append(img_data["label"])
    break
output_dict

{'img': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x7F456D632380>, 'label': 0}
On idx 0


OrderedDict([('obj_ids',
              tensor([[ 106,   72,  395,   72,  106,  364,  364,  364,  395,  956,  106,  106,
                        395,  106,  395,  106,  956,   71,   72,  395,  364,  956,  395,   72,
                        106,  395,  395,  395,   72,  106,  956,  177,  106,  242,  395, 1547]])),
             ('obj_probs',
              tensor([[0.4995, 0.4922, 0.4374, 0.4207, 0.4112, 0.4067, 0.3885, 0.3468, 0.3144,
                       0.2768, 0.2723, 0.2403, 0.2272, 0.2226, 0.2132, 0.2031, 0.1965, 0.1931,
                       0.1884, 0.1879, 0.1859, 0.1817, 0.1760, 0.1739, 0.1732, 0.1623, 0.1539,
                       0.1515, 0.1485, 0.1362, 0.1222, 0.1221, 0.1163, 0.1112, 0.1088, 0.0892]])),
             ('attr_ids',
              tensor([[210,   7, 210,   7, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210,
                       210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210,
                       210, 210, 210, 210, 210, 210, 210, 210]]

In [13]:
output_dict['normalized_boxes'].shape

torch.Size([1, 36, 4])

In [14]:
print("Generating " + str(len(train_data)) + " embeddings")

visual_embeddings = []
labels = []
bounding_boxes = []

for idx, img_data in enumerate(train_data):
    if idx % 1000 == 0:
        print("On idx " + str(idx))
    image = np.array(img_data["img"])
    # run frcnn
    images, sizes, scales_yx = image_preprocess(image)
    output_dict = frcnn(
        images,
        sizes,
        scales_yx=scales_yx,
        padding="max_detections",
        max_detections=frcnn_cfg.max_detections,
        return_tensors="pt",
    )
    features = output_dict.get("roi_features")
    box = output_dict.get("normalized_boxes")
    
    visual_embeddings.append(features)
    bounding_boxes.append(box)
    labels.append(img_data["label"])


Generating 50000 embeddings
On idx 0
On idx 1000
On idx 2000
On idx 3000
On idx 4000
On idx 5000
On idx 6000
On idx 7000
On idx 8000
On idx 9000
On idx 10000
On idx 11000
On idx 12000
On idx 13000
On idx 14000
On idx 15000
On idx 16000
On idx 17000
On idx 18000
On idx 19000
On idx 20000
On idx 21000
On idx 22000
On idx 23000
On idx 24000
On idx 25000
On idx 26000
On idx 27000
On idx 28000
On idx 29000
On idx 30000
On idx 31000
On idx 32000
On idx 33000
On idx 34000
On idx 35000
On idx 36000
On idx 37000
On idx 38000
On idx 39000
On idx 40000
On idx 41000
On idx 42000
On idx 43000
On idx 44000
On idx 45000
On idx 46000
On idx 47000
On idx 48000
On idx 49000


In [15]:
cifar10_train_embeddings = {
    "embeddings": visual_embeddings,
    "bounding_box": bounding_boxes,
    "labels": labels
}

with open("cifar10-train-embeddings.pkl", "wb") as f:
    pickle.dump(cifar10_train_embeddings, f)

In [16]:
print("Generating " + str(len(test_data)) + " embeddings")

visual_embeddings = []
labels = []
bounding_boxes = []

for idx, img_data in enumerate(test_data):
    if idx % 1000 == 0:
        print("On idx " + str(idx))
    image = np.array(img_data["img"])
    # run frcnn
    images, sizes, scales_yx = image_preprocess(image)
    output_dict = frcnn(
        images,
        sizes,
        scales_yx=scales_yx,
        padding="max_detections",
        max_detections=frcnn_cfg.max_detections,
        return_tensors="pt",
    )
    features = output_dict.get("roi_features")
    box = output_dict.get("normalized_boxes")
    
    visual_embeddings.append(features)
    bounding_boxes.append(box)
    labels.append(img_data["label"])


Generating 10000 embeddings
On idx 0
On idx 1000
On idx 2000
On idx 3000
On idx 4000
On idx 5000
On idx 6000
On idx 7000
On idx 8000
On idx 9000


In [17]:
cifar10_test_embeddings = {
    "embeddings": visual_embeddings,
    "bounding_box": bounding_boxes,
    "labels": labels
}

with open("cifar10-test-embeddings.pkl", "wb") as f:
    pickle.dump(cifar10_test_embeddings, f)